# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [12]:
def retrieve_titles(soup):
    book_container = soup.find('div', class_= 'alert alert-warning').nextSibling.nextSibling
    titles = [h3.find('a').attrs['title'] for h3 in book_container.findAll('h3')]
    return titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [14]:
def retrieve_ratings(soup):
    book_container = soup.find('div', class_= 'alert alert-warning').nextSibling.nextSibling
    regex = re.compile("star-rating (.*)")
    star_ratings = []
    for p in book_container.findAll('p', {'class': regex}):
        star_ratings.append(p.attrs['class'][-1])
    return star_ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [3]:
def retrieve_prices(soup):
    book_container = soup.find('div', class_= 'alert alert-warning').nextSibling.nextSibling
    prices = [p.text for p in book_container.findAll('p', class_= 'price_color')]
    prices = [float(p[1:]) for p in prices]
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [43]:
def retrieve_availabilities(soup):
    book_container = soup.find('div', class_= 'alert alert-warning').nextSibling.nextSibling
    avails = [a.text.strip() for a in book_container.findAll('p', class_="instock availability")]
    return avails

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [44]:
#RUN THE CELL BELOW FIRST TO MAKE SURE LIBRARIES ARE IMPORTED
def initiate():
    html_page = requests.get("http://books.toscrape.com/catalogue/page-1.html")
    soup = BeautifulSoup(html_page.content, 'html.parser')
    titles = retrieve_titles(soup)
    ratings = retrieve_ratings(soup)
    prices = retrieve_prices(soup)
    avails = retrieve_availabilities(soup)
    df = pd.DataFrame([titles, ratings, prices, avails]).transpose()
    df.columns = ['Title', 'Rating', 'Price in Pounds', 'Availability']
    return df

In [49]:
df = initiate()

In [46]:
df

,Title,Rating,Price in Pounds,Availability
0,A Light in the Attic,Three,51.77,In stock
1,Tipping the Velvet,One,53.74,In stock
2,Soumission,One,50.1,In stock
3,Sharp Objects,Four,47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock
5,The Requiem Red,One,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,Four,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,Three,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,Four,22.6,In stock
9,The Black Maria,One,52.15,In stock


In [50]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
for i in range(2, 51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    titles = retrieve_titles(soup)
    ratings = retrieve_ratings(soup)
    prices = retrieve_prices(soup)
    avails = retrieve_availabilities(soup)
    df_temp = pd.DataFrame([titles, ratings, prices, avails]).transpose()
    df_temp.columns = ['Title', 'Rating', 'Price in Pounds', 'Availability']
    df = df.append(df_temp)

In [51]:
df

,Title,Rating,Price in Pounds,Availability
0,A Light in the Attic,Three,51.77,In stock
1,Tipping the Velvet,One,53.74,In stock
2,Soumission,One,50.1,In stock
3,Sharp Objects,Four,47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock
...,...,...,...,...
15,Alice in Wonderland (Alice's Adventures in Won...,One,55.53,In stock
16,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,57.06,In stock
17,A Spy's Devotion (The Regency Spies of London #1),Five,16.97,In stock
18,1st to Die (Women's Murder Club #1),One,53.98,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [24]:
for i in range(2, 3):
    print(i)

2


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!